In [ ]:

# 중도탈락학생(신입생)비율(%) 데이터 전처리 템플릿

import pandas as pd

#  파일 경로 설정
file_path = "파일명을 여기에 입력하세요.xlsx"

#  데이터 로딩 (앞쪽 헤더 4줄은 무시)
df = pd.read_excel(file_path, skiprows=4)

#  연도 및 학교명, 중도탈락(신입생)비율 추출
df_filtered = pd.DataFrame({
    "기준연도": df["Unnamed: 0"].fillna(method='ffill'),
    "학교명": df["Unnamed: 5"],
    "중도탈락학생(신입생)비율(%)": df["Unnamed: 27"]
})

#  결측치 제거
df_filtered.dropna(subset=["기준연도", "학교명", "중도탈락학생(신입생)비율(%)"], inplace=True)

#  연도 정수형 변환
df_filtered["기준연도"] = df_filtered["기준연도"].astype(int)

#  저장
df_filtered.to_csv("중도탈락학생_신입생비율_출력파일.csv", index=False, encoding="utf-8-sig")

#  결과 확인
df_filtered.head()


In [ ]:

# 신입생 기준 중도탈락율(2020~2023) 전처리 템플릿
# 대상: 2023년 기준 존재하는 학교 + 사이버대학 제외

import pandas as pd

#  각 연도별 CSV 파일 로드
df_2020 = pd.read_csv("중도탈락학생_신입생비율_2020_UTF8BOM.csv")
df_2021 = pd.read_csv("중도탈락학생_신입생비율_2021_UTF8BOM.csv")
df_2022 = pd.read_csv("중도탈락학생_신입생비율_2022_UTF8BOM.csv")
df_2023 = pd.read_csv("중도탈락학생_신입생비율_2023_UTF8BOM.csv")

#  통합 및 컬럼명 정리
df_all = pd.concat([df_2020, df_2021, df_2022, df_2023])
df_all["기준연도"] = df_all["기준연도"].astype(int)
df_all.rename(columns={"중도탈락학생(신입생)비율(%)": "중도탈락율"}, inplace=True)

# 2023 기준 학교 목록 추출
schools_2023 = df_all[df_all["기준연도"] == 2023]["학교명"].unique()

#  '사이버' 포함된 학교 제거
filtered_schools = [s for s in schools_2023 if "사이버" not in s]

#  최종 필터링: 2023 존재 + 사이버학교 제외
df_filtered = df_all[df_all["학교명"].isin(filtered_schools)].copy()

#  결과 저장
df_filtered.to_csv("신입생중도탈락율_2020_2023_사이버대학제외_2023기준학교만.csv", index=False, encoding="utf-8-sig")

#  연도별 학교 수 확인
df_filtered.groupby("기준연도")["학교명"].nunique()


In [ ]:

#  신입생 기준 중도탈락율 병합 중간 단계 전처리 템플릿

import pandas as pd
from scipy.stats import zscore

#  전처리된 중도탈락율 데이터 로드
df_dropout = pd.read_csv("신입생중도탈락율_2020_2023_사이버대학제외_2023기준학교만.csv")
df_dropout.rename(columns={"중도탈락율": "중도탈락학생(신입생)비율(%)"}, inplace=True)
df_dropout["학교명"] = df_dropout["학교명"].str.strip()

#  Z-score + 0값 유지 + 2020~2023 공통학교 파일 로드
df_zscore = pd.read_csv("최종통합_2020_2023_2023기준학교만_Zscore_0포함_공통학교.csv")
df_zscore["학교명"] = df_zscore["학교명"].str.strip()

# 중도탈락율 제거 후 신입생 기준 비율로 병합
df_zscore_merged = df_zscore.drop(columns=["중도탈락율"], errors="ignore")
df_zscore_merged = df_zscore_merged.merge(df_dropout, how="inner", on=["기준연도", "학교명"])

#  2020~2022 기준 공통 173개 학교만 유지
common_173 = set.intersection(*[
    set(df_zscore_merged[df_zscore_merged["기준연도"] == y]["학교명"]) for y in range(2020, 2023)
])
df_zscore_173 = df_zscore_merged[df_zscore_merged["학교명"].isin(common_173)].copy()

# 저장
df_zscore_173.to_csv("신입생기준_Zscore_0포함_공통학교_173개_통일.csv", index=False, encoding="utf-8-sig")

# 확인
print(df_zscore_173.groupby("기준연도")["학교명"].nunique())


In [ ]:

#  대학 중도탈락율 전처리 전체 흐름 (신입생 기준, 2020~2023)
# 목적: 선형회귀 모델 학습용 정제된 데이터셋 생성 (Z-score 이상치 제거 포함, 사이버대학 제외, 2023 기준)

import pandas as pd
from scipy.stats import zscore

#  1. 신입생 기준 중도탈락율 데이터 불러오기 (전처리된 2020~2023 통합본)
df_dropout = pd.read_csv("신입생중도탈락율_2020_2023_사이버대학제외_2023기준학교만.csv")
df_dropout.rename(columns={"중도탈락율": "중도탈락학생(신입생)비율(%)"}, inplace=True)
df_dropout["학교명"] = df_dropout["학교명"].str.strip()

#  2. 통합 피처 데이터 불러오기 (예: 최종 통합파일, 연도 2020~2023, 2023 기준학교만)
df_all = pd.read_csv("최종통합_2020_2023_2023기준학교만.csv")
df_all.rename(columns={"기준년도": "기준연도"}, inplace=True)
df_all["학교명"] = df_all["학교명"].str.strip()

#  3. 결측치 제거 버전 생성
df_cleaned = df_all.dropna().copy()
df_cleaned = df_cleaned.drop(columns=["중도탈락율"], errors="ignore")
df_cleaned = df_cleaned.merge(df_dropout, how="inner", on=["기준연도", "학교명"])
df_cleaned.to_csv("신입생기준_결측치제거_통합데이터.csv", index=False, encoding="utf-8-sig")

#  4. Z-score 이상치 제거 버전 (0값은 유지, ±3 이상만 제거)
numeric_cols = df_all.select_dtypes(include=["float64", "int64"]).columns.difference(["기준연도"])
def zscore_filter_keep_zero(group):
    zscores = zscore(group[numeric_cols], nan_policy="omit")
    mask = ~((abs(zscores) >= 3) & (group[numeric_cols] != 0)).any(axis=1)
    return group[mask]

df_zscore = df_all.groupby("기준연도", group_keys=False).apply(zscore_filter_keep_zero)

#  5. 공통으로 존재하는 학교만 필터링 (2020~2023 연도 모두에 있는 학교 기준)
common_schools = set.intersection(*[
    set(df_zscore[df_zscore["기준연도"] == y]["학교명"].unique()) for y in range(2020, 2024)
])
df_zscore_common = df_zscore[df_zscore["학교명"].isin(common_schools)].copy()

#  6. 2020~2022 기준 공통된 학교(173개)만 유지 (영산대학교(양산) _제2캠퍼스 제외)
common_173 = set.intersection(*[
    set(df_zscore_common[df_zscore_common["기준연도"] == y]["학교명"].unique()) for y in range(2020, 2023)
])
df_zscore_173 = df_zscore_common[df_zscore_common["학교명"].isin(common_173)].copy()

#  7. 신입생 기준 중도탈락율로 병합 및 저장
df_zscore_173 = df_zscore_173.drop(columns=["중도탈락율"], errors="ignore")
df_zscore_173 = df_zscore_173.merge(df_dropout, how="inner", on=["기준연도", "학교명"])
df_zscore_173.to_csv("신입생기준_Zscore_0포함_공통학교_173개_통일.csv", index=False, encoding="utf-8-sig")

#  확인
print(df_cleaned.groupby("기준연도")["학교명"].nunique())
print(df_zscore_173.groupby("기준연도")["학교명"].nunique())
